# Chunker

The syntax of a natural language, similar to the syntax of a programming language involves the arrangement of tokens into meaningful groups. Phrasal chunking is the task of finding non-recursive syntactic groups of words.We explore different techniques to accomplish this task in this assignment

In [12]:
from chunker import *
import os
import logging

import conlleval

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Imports done successfully")

DATA = '../data'

INFO:root:Imports done successfully


# 1. Baseline model

**The default solution given for this assignment provides us a score of 72.** In order to improve the score we design our baseline model according to the following logic.

We build a semi-character Recurrent Neural Network as described by the paper titled _"Robsut Wrod Reocginiton via Semi-Character Recurrent Neural Network"._ The idea presented in this paper is inspired from the <I>Cmabrigde Uinervtisy</I> effect where the word recognition model is based on semi-character level recurrent neural network.

The schematic illustraion of semi-character recurrent neural network is as follows:
<br>
<br>
<img src="img.png">
<br>
<br>
The following method was used to create a character level representation of the word:
1. A one-hot vector "beginning" for the first character of the word.
2. A vector "internal" where the index of a character has the count of that character in the word.
3. A one-hot vector "ending" for the last character of the word.

Eventually we concatenated to the word embedding input to the chunker RNN an input vector that is the character level representation of the word.

**This gave us a score of 77.4683**

Thus our baseline model improves the score of chunker to 77.4683. Modifications were done to the LSTMTaggerModel and LSTMTagger classes. The code and the output is given below:

In [18]:
class LSTMTaggerModel(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        torch.manual_seed(1)
        super(LSTMTaggerModel, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(428, hidden_dim, bidirectional=False)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence_in, sentence):
        embeds = self.word_embeddings(sentence_in)
        stack_ = []
        for word in sentence:
            beginning = {}
            internal = {}
            ending = {}
            for char in string.printable:
                beginning[char] = 0
                internal [char] = 0
                ending[char] = 0
            length = len(word)
            b = word[0]
            e = word [length-1]
            for char in word[1:length-1]:
                 internal[char] = internal.get(char) + 1
            beginning[b] = beginning.get(b) + 1
            ending[e] = ending.get(e) + 1
            
            beginning_vect = []
            k = list(beginning.keys()) 
            k.sort()
            for i in k:
                beginning_vect.append(beginning[i])
    
            internal_vect = []    
            k = list(internal.keys())
            k.sort()
            for i in k:
                internal_vect.append(internal[i])
 
            ending_vect = []
            k = list(ending.keys())
            k.sort()
            for i in k:
                ending_vect.append(ending[i])

            beginning_tensor = torch.FloatTensor(beginning_vect)
            internal_tensor = torch.FloatTensor(internal_vect)
            ending_tensor = torch.FloatTensor(ending_vect)
            
            
            
            character_level = torch.cat((beginning_tensor, internal_tensor, ending_tensor), 0)
            
            stack_.append(character_level)
        
        stack_tensor = torch.stack(stack_, dim = 0)    

        embeds = torch.cat((embeds, stack_tensor), 1)   
        lstm_out, _ = self.lstm(embeds.view(len(sentence_in), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence_in), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
    
class LSTMTagger:

    def __init__(self, trainfile, modelfile, modelsuffix, unk="[UNK]", epochs=10, embedding_dim=128, hidden_dim=64):
        self.unk = unk
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.epochs = epochs
        self.modelfile = modelfile
        self.modelsuffix = modelsuffix
        self.training_data = []
        if trainfile[-3:] == '.gz':
            with gzip.open(trainfile, 'rt') as f:
                self.training_data = read_conll(f)
        else:
            with open(trainfile, 'r') as f:
                self.training_data = read_conll(f)

        self.word_to_ix = {} # replaces words with an index (one-hot vector)
        self.tag_to_ix = {} # replace output labels / tags with an index
        self.ix_to_tag = [] # during inference we produce tag indices so we have to map it back to a tag

        for sent, tags in self.training_data:
            for word in sent:
                if word not in self.word_to_ix:
                    self.word_to_ix[word] = len(self.word_to_ix)
            for tag in tags:
                if tag not in self.tag_to_ix:
                    self.tag_to_ix[tag] = len(self.tag_to_ix)
                    self.ix_to_tag.append(tag)

        logging.info("word_to_ix:", self.word_to_ix)
        logging.info("tag_to_ix:", self.tag_to_ix)
        logging.info("ix_to_tag:", self.ix_to_tag)

        self.model = LSTMTaggerModel(self.embedding_dim, self.hidden_dim, len(self.word_to_ix), len(self.tag_to_ix))
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.01)

    def argmax(self, seq):
        output = []
        with torch.no_grad():
            inputs = prepare_sequence(seq, self.word_to_ix, self.unk)
            tag_scores = self.model(inputs, seq)
            for i in range(len(inputs)):
                output.append(self.ix_to_tag[int(tag_scores[i].argmax(dim=0))])
        return output

    def train(self):
        loss_function = nn.NLLLoss()

        self.model.train()
        loss = float("inf")
        for epoch in range(self.epochs):
            for sentence, tags in tqdm.tqdm(self.training_data):
                # Step 1. Remember that Pytorch accumulates gradients.
                # We need to clear them out before each instance
                self.model.zero_grad()

                # Step 2. Get our inputs ready for the network, that is, turn them into
                # Tensors of word indices.
                sentence_in = prepare_sequence(sentence, self.word_to_ix, self.unk)
                targets = prepare_sequence(tags, self.tag_to_ix, self.unk)

                # Step 3. Run our forward pass.
                tag_scores = self.model(sentence_in, sentence)

                # Step 4. Compute the loss, gradients, and update the parameters by
                #  calling optimizer.step()
                loss = loss_function(tag_scores, targets)
                loss.backward()
                self.optimizer.step()

            if epoch == self.epochs-1:
                epoch_str = '' # last epoch so do not use epoch number in model filename
            else:
                epoch_str = str(epoch)
            savefile = self.modelfile + epoch_str + self.modelsuffix
            print("saving model file: {}".format(savefile), file=sys.stderr)
            torch.save({
                        'epoch': epoch,
                        'model_state_dict': self.model.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'loss': loss,
                        'unk': self.unk,
                        'word_to_ix': self.word_to_ix,
                        'tag_to_ix': self.tag_to_ix,
                        'ix_to_tag': self.ix_to_tag,
                    }, savefile)

    def decode(self, inputfile):
        if inputfile[-3:] == '.gz':
            with gzip.open(inputfile, 'rt') as f:
                input_data = read_conll(f, input_idx=0, label_idx=-1)
        else:
            with open(inputfile, 'r') as f:
                input_data = read_conll(f, input_idx=0, label_idx=-1)

        if not os.path.isfile(self.modelfile + self.modelsuffix):
            raise IOError("Error: missing model file {}".format(self.modelfile + self.modelsuffix))

        saved_model = torch.load(self.modelfile + self.modelsuffix)
        self.model.load_state_dict(saved_model['model_state_dict'])
        self.optimizer.load_state_dict(saved_model['optimizer_state_dict'])
        epoch = saved_model['epoch']
        loss = saved_model['loss']
        self.unk = saved_model['unk']
        self.word_to_ix = saved_model['word_to_ix']
        self.tag_to_ix = saved_model['tag_to_ix']
        self.ix_to_tag = saved_model['ix_to_tag']
        self.model.eval()
        decoder_output = []
        for sent in tqdm.tqdm(input_data):
            decoder_output.append(self.argmax(sent))
        return decoder_output

## Run and evaluate the baseline model

In [ ]:
chunker = LSTMTagger(os.path.join(DATA, 'train.txt.gz'), os.path.join(DATA, 'chunker'), '.tar')
decoder_output = chunker.decode(DATA + '/input/dev.txt')

In [21]:
flat_output = [ output for sent in decoder_output for output in sent ]
true_seqs = []
with open(os.path.join(DATA,'reference','dev.out')) as r:
    for sent in conlleval.read_file(r):
        true_seqs += sent.split()
conlleval.evaluate(true_seqs, flat_output)

processed 23663 tokens with 11896 phrases; found: 11977 phrases; correct: 9247.
accuracy:  86.89%; (non-O)
accuracy:  87.91%; precision:  77.21%; recall:  77.73%; FB1:  77.47
             ADJP: precision:  50.00%; recall:  19.91%; FB1:  28.48  90
             ADVP: precision:  68.68%; recall:  45.73%; FB1:  54.90  265
            CONJP: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
             INTJ: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
               NP: precision:  75.35%; recall:  81.18%; FB1:  78.16  6719
               PP: precision:  92.21%; recall:  86.81%; FB1:  89.43  2298
              PRT: precision:  70.00%; recall:  62.22%; FB1:  65.88  40
             SBAR: precision:  82.81%; recall:  44.73%; FB1:  58.08  128
               VP: precision:  69.92%; recall:  73.96%; FB1:  71.88  2437


(77.20631209818819, 77.7320107599193, 77.46826959326437)

# 2. Further improvements to the baseline model

After implementing one-hot vectors as explained in "Robsut Wrod Reocginiton via Semi-Character Recurrent Neural Network", we trained another model using word embeddings and character embeddings together.

Previously, we were encoding three vectors for each word, i.e. beginning, internal and ending. That approach has some flaws as shown with this example.

**Assume two valid English words that are also anagrams; "placebo" and "obecalp".**

internal vector representation for both these words will be the same and beginning and ending vectors will be swapped. Since the model doesn't treat these one-hot vectors as parameters, it never learns that the slice "laceb" in "placebo" is different than "becalp" in "obecalp" and treats both occurences as same. Introducing character embeddings that are also learnt during the training process avoids this disambiguation.

We created a character embedding with an input size of 100, length of string.printable. Embedding dimensions were also kept 100 for easier matrix multiplications.

During the forward pass, we create one-hot vectors just like in the previous section. After that, we multiply these one-hot vectors with the character embeddings as follows:

```
beginning: matrix of dim [1 x 100] * matrix of dim [100 x 100]
internal: matrix of dim [1 x 100] * matrix of dim [100 x 100]
ending: matrix of dim [1 x 100] * matrix of dim [100 x 100]
```

The result is three vectors of dim 100 each which are then concatenated with the word embeddings.

The following code shows the change we made to our baseline, the training process and the **final FB1 score of 79.1829 that we attained from this model**.

In [116]:
from collections import OrderedDict


class LSTMTaggerModel(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        torch.manual_seed(1)
        super(LSTMTaggerModel, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # Character embeddings have dimension of 100
        self.char_embeddings = nn.Embedding(len(string.printable), len(string.printable))

        # Build character indexes
        self.char_idx = OrderedDict()
        for c in string.printable:
            if c not in self.char_idx:
                self.char_idx[c] = len(self.char_idx)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(428, hidden_dim, bidirectional=False)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
    
    def _word_to_char_idx(self, word):
        # Return indices for character embedding lookup. Returns 100 (default) for unknown character
        indices = [self.char_idx[c] for c in word]
        return indices
    
    def forward(self, sentence_in, sentence):
        embeds = self.word_embeddings(sentence_in)
        stack_ = []
        for word in sentence:
            # Get embeddings for all characters
            char_indices = torch.tensor([self.char_idx[c] for c in string.printable], dtype=torch.long)
            char_embeds = self.char_embeddings(char_indices)
            
            beginning = {}
            internal = {}
            ending = {}
            for char in string.printable:
                beginning[char] = 0
                internal [char] = 0
                ending[char] = 0
            length = len(word)
            b = word[0]
            e = word [length-1]
            for char in word[1:length-1]:
                 internal[char] = internal.get(char) + 1
            beginning[b] = beginning.get(b) + 1
            ending[e] = ending.get(e) + 1
            
            beginning_vect = []
            k = list(beginning.keys()) 
            k.sort()
            for i in k:
                beginning_vect.append(beginning[i])
    
            internal_vect = []    
            k = list(internal.keys())
            k.sort()
            for i in k:
                internal_vect.append(internal[i])
 
            ending_vect = []
            k = list(ending.keys())
            k.sort()
            for i in k:
                ending_vect.append(ending[i])
            
            # All of same size in this case
            vector_size = len(beginning_vect)
            
            # Need to reshape one-hot matrix in order to do multiplications
            beginning_tensor = torch.FloatTensor(beginning_vect).reshape(1, vector_size)
            beginning_tensor = torch.mm(beginning_tensor, char_embeds)[0]
            
            internal_tensor = torch.FloatTensor(internal_vect).reshape(1, vector_size)
            internal_tensor = torch.mm(internal_tensor, char_embeds)[0]
            
            ending_tensor = torch.FloatTensor(ending_vect).reshape(1, vector_size)
            ending_tensor = torch.mm(ending_tensor, char_embeds)[0]
            
            character_level = torch.cat((beginning_tensor, internal_tensor, ending_tensor), 0)
            
            stack_.append(character_level)
        
        stack_tensor = torch.stack(stack_, dim = 0)    

        embeds = torch.cat((embeds, stack_tensor), 1)   
        lstm_out, _ = self.lstm(embeds.view(len(sentence_in), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence_in), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
    

class LSTMTagger:

    def __init__(self, trainfile, modelfile, modelsuffix, unk="[UNK]", epochs=10, embedding_dim=128, hidden_dim=64):
        self.unk = unk
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.epochs = epochs
        self.modelfile = modelfile
        self.modelsuffix = modelsuffix
        self.training_data = []
        if trainfile[-3:] == '.gz':
            with gzip.open(trainfile, 'rt') as f:
                self.training_data = read_conll(f)
        else:
            with open(trainfile, 'r') as f:
                self.training_data = read_conll(f)

        self.word_to_ix = {} # replaces words with an index (one-hot vector)
        self.tag_to_ix = {} # replace output labels / tags with an index
        self.ix_to_tag = [] # during inference we produce tag indices so we have to map it back to a tag

        for sent, tags in self.training_data:
            for word in sent:
                if word not in self.word_to_ix:
                    self.word_to_ix[word] = len(self.word_to_ix)
            for tag in tags:
                if tag not in self.tag_to_ix:
                    self.tag_to_ix[tag] = len(self.tag_to_ix)
                    self.ix_to_tag.append(tag)

        logging.info("word_to_ix:", self.word_to_ix)
        logging.info("tag_to_ix:", self.tag_to_ix)
        logging.info("ix_to_tag: %s", self.ix_to_tag)

        self.model = LSTMTaggerModel(self.embedding_dim, self.hidden_dim, len(self.word_to_ix), len(self.tag_to_ix))
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.01)

    def argmax(self, seq):
        output = []
        with torch.no_grad():
            inputs = prepare_sequence(seq, self.word_to_ix, self.unk)
            tag_scores = self.model(inputs, seq)
            for i in range(len(inputs)):
                output.append(self.ix_to_tag[int(tag_scores[i].argmax(dim=0))])
        return output

    def train(self):
        loss_function = nn.NLLLoss()

        self.model.train()
        loss = float("inf")
        for epoch in range(self.epochs):
            for sentence, tags in tqdm.tqdm(self.training_data):
                # Step 1. Remember that Pytorch accumulates gradients.
                # We need to clear them out before each instance
                self.model.zero_grad()

                # Step 2. Get our inputs ready for the network, that is, turn them into
                # Tensors of word indices.
                sentence_in = prepare_sequence(sentence, self.word_to_ix, self.unk)
                targets = prepare_sequence(tags, self.tag_to_ix, self.unk)

                # Step 3. Run our forward pass.
                tag_scores = self.model(sentence_in, sentence)

                # Step 4. Compute the loss, gradients, and update the parameters by
                #  calling optimizer.step()
                loss = loss_function(tag_scores, targets)
                loss.backward()
                self.optimizer.step()

            if epoch == self.epochs-1:
                epoch_str = '' # last epoch so do not use epoch number in model filename
            else:
                epoch_str = str(epoch)
            savefile = self.modelfile + epoch_str + self.modelsuffix
            print("saving model file: {}".format(savefile), file=sys.stderr)
            torch.save({
                        'epoch': epoch,
                        'model_state_dict': self.model.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'loss': loss,
                        'unk': self.unk,
                        'word_to_ix': self.word_to_ix,
                        'tag_to_ix': self.tag_to_ix,
                        'ix_to_tag': self.ix_to_tag,
                    }, savefile)

    def decode(self, inputfile):
        if inputfile[-3:] == '.gz':
            with gzip.open(inputfile, 'rt') as f:
                input_data = read_conll(f, input_idx=0, label_idx=-1)
        else:
            with open(inputfile, 'r') as f:
                input_data = read_conll(f, input_idx=0, label_idx=-1)

        if not os.path.isfile(self.modelfile + self.modelsuffix):
            raise IOError("Error: missing model file {}".format(self.modelfile + self.modelsuffix))

        saved_model = torch.load(self.modelfile + self.modelsuffix)
        self.model.load_state_dict(saved_model['model_state_dict'])
        self.optimizer.load_state_dict(saved_model['optimizer_state_dict'])
        epoch = saved_model['epoch']
        loss = saved_model['loss']
        self.unk = saved_model['unk']
        self.word_to_ix = saved_model['word_to_ix']
        self.tag_to_ix = saved_model['tag_to_ix']
        self.ix_to_tag = saved_model['ix_to_tag']
        self.model.eval()
        decoder_output = []
        for sent in tqdm.tqdm(input_data):
            decoder_output.append(self.argmax(sent))
        return decoder_output

In [117]:
logger.setLevel(logging.ERROR)
chunker = LSTMTagger(os.path.join(DATA, 'train.txt.gz'), os.path.join(DATA, 'chunkerTres'), '.tar')
chunker.train()

100%|██████████| 8936/8936 [04:07<00:00, 36.07it/s]
saving model file: ../data/chunkerTres0.tar
100%|██████████| 8936/8936 [03:57<00:00, 37.68it/s]
saving model file: ../data/chunkerTres1.tar
100%|██████████| 8936/8936 [03:58<00:00, 37.39it/s]
saving model file: ../data/chunkerTres2.tar
100%|██████████| 8936/8936 [03:59<00:00, 37.35it/s]
saving model file: ../data/chunkerTres3.tar
100%|██████████| 8936/8936 [04:10<00:00, 35.65it/s]
saving model file: ../data/chunkerTres4.tar
100%|██████████| 8936/8936 [03:56<00:00, 37.81it/s]
saving model file: ../data/chunkerTres5.tar
100%|██████████| 8936/8936 [04:34<00:00, 32.52it/s]
saving model file: ../data/chunkerTres6.tar
100%|██████████| 8936/8936 [04:27<00:00, 33.38it/s]
saving model file: ../data/chunkerTres7.tar
100%|██████████| 8936/8936 [04:24<00:00, 33.75it/s]
saving model file: ../data/chunkerTres8.tar
100%|██████████| 8936/8936 [03:57<00:00, 37.62it/s]
saving model file: ../data/chunkerTres.tar


In [118]:
decoder_output = chunker.decode(DATA + '/input/dev.txt')

100%|██████████| 1027/1027 [00:05<00:00, 187.21it/s]


In [119]:
flat_output = [ output for sent in decoder_output for output in sent ]
true_seqs = []
with open(os.path.join(DATA,'reference','dev.out')) as r:
    for sent in conlleval.read_file(r):
        true_seqs += sent.split()
conlleval.evaluate(true_seqs, flat_output)

processed 23663 tokens with 11896 phrases; found: 12483 phrases; correct: 9652.
accuracy:  87.95%; (non-O)
accuracy:  88.96%; precision:  77.32%; recall:  81.14%; FB1:  79.18
             ADJP: precision:  49.12%; recall:  24.78%; FB1:  32.94  114
             ADVP: precision:  63.23%; recall:  57.04%; FB1:  59.97  359
            CONJP: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
             INTJ: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
               NP: precision:  77.05%; recall:  83.89%; FB1:  80.33  6790
               PP: precision:  91.32%; recall:  88.73%; FB1:  90.01  2372
              PRT: precision:  65.91%; recall:  64.44%; FB1:  65.17  44
             SBAR: precision:  73.08%; recall:  48.10%; FB1:  58.02  156
               VP: precision:  69.03%; recall:  79.34%; FB1:  73.83  2648


(77.32115677321157, 81.13651647612643, 79.1829033184298)

# 3. Additional experiments tried and their evaluation

We experimented with different approaches by utilizing a better optimizer. Our original solution used Stochastic Gradient Descent. We had an intuition that it may not be converging quickly enough and led to suboptimal results. 

Figure comparing gradient descent v/s gradient descent with momentum.
<br>
<br>
<img src="img1.png">
<br>
<br>

Looking at the figure we can see that using momentum can lead to faster convergence.

We tried the following optimizers:

1. Adam
2. Stochastic Gradient Descent with momentum
3. RMSProp

#### Changed code snippets in LSTMTagger for training:

```python
self.optimizer = optim.SGD(self.model.parameters(), lr=0.01)
self.optimizer = optim.SGD(self.model.parameters(), lr=0.01, momentum = 0.9)
self.optimizer = optim.RMSprop(self.model.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)
```

#### We got the following results:

| Optimizer | Dev.out Score |
| --- | --- |
| SGD | 77.4683 |
| Adam | ~75 |
| SGD with momentum | 79.3922 |
| RMSprop | 78.6700 |

We can see that we have got a significant boost in the dev.out score using SGD with momentum and RMSprop. However, this violates the requirement of the assignment so this wasn't part of our final submission.

# 4. Final model and conclusion

Our final model is based on word embeddings and character embeddings as explained in section 3 of the notebook. Note that we haven't changed any hyperparameters or optimizers in this model which was one of the requirements of this assignment.

Our model achieved a FB1 score of **79.1829**